# Stress Analysis in Social Media - Innovation

Shir Cohen 205790124

Guy Assa 204118616




## Initialisation 

### Imports

In [ ]:
import torch
torch.cuda.empty_cache()
CUDA_LAUNCH_BLOCKING=1

In [ ]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 3.2 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3165914 sha256=a38fc89fc65a7277b903e2c0fcd5149788f0c19c112a3f67aa5fd7072b79cf81
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
import torch
import pandas as pd
import numpy as np
import os
import string 
import tqdm
import warnings
import logging # This allows for seeing if the model converges. A log file is created.
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import re

# from spellchecker import SpellChecker

from gensim.models import LdaModel
from gensim import models, corpora, similarities
import gensim
from gensim.utils import simple_preprocess

from keras.layers import LSTM, Activation, Dropout, Dense, Input
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Embedding,CuDNNLSTM,Bidirectional

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.feature_extraction import text

from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import fasttext.util

In [ ]:
# ------------ https://towardsdatascience.com/unsupervised-nlp-topic-models-as-a-supervised-learning-input-cf8ee9e5cf28
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove.6B.zip

In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


### Data

In [ ]:
df_train = pd.read_csv("dreaddit-train.csv", index_col = 0)
df_test = pd.read_csv("dreaddit-test.csv", index_col = 0)

In [ ]:
train = df_train.drop(['post_id', 'sentence_range', 'id'], axis = 1)
test = df_test.drop(['post_id', 'sentence_range', 'id'], axis = 1)

## Preprocessing

In [ ]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [ ]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    # Tokenize each word
    text =  nltk.WordPunctTokenizer().tokenize(text)
        
    return text

In [ ]:
def normalize(s):
    """
    Given a text, cleans and normalizes it.
    """
    s = s.lower()
    # Replace ips
    s = re.sub(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', ' _ip_ ', s)
    # Isolate punctuation
    s = re.sub(r'([\'\"\.\(\)\!\?\-\\\/\,])', r' \1 ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Replace numbers and symbols with language
    s = s.replace('&', ' and ')
    s = s.replace('@', ' at ')
    s = s.replace('0', ' zero ')
    s = s.replace('1', ' one ')
    s = s.replace('2', ' two ')
    s = s.replace('3', ' three ')
    s = s.replace('4', ' four ')
    s = s.replace('5', ' five ')
    s = s.replace('6', ' six ')
    s = s.replace('7', ' seven ')
    s = s.replace('8', ' eight ')
    s = s.replace('9', ' nine ')
    return s

def text_preprocessing(s):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    - Lowercase the sentence
    - Change "'t" to "not"
    - Remove "@name"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    """
    s = s.lower()

    s = s.split()
    new_text = []
    for word in s:
      try:
        new_text.append(contractions.fix(word))
      except:
        new_text.append(word)
    s = " ".join(new_text)

    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove stopwords except 'not' and 'can'
    s = " ".join([word for word in s.split()
                  if word not in stopwords.words('english')
                  or word in ['not', 'can']])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    # Replace '&amp;' with '&'
    setattr = re.sub(r'&amp;', '&', s)
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    s = re.sub(r'https?:\/\/.*[\r\n]*', '', s, flags=re.MULTILINE)
    slice = re.sub(r'\<a href', ' ', s)
    s = re.sub(r'&amp;', '', s) 
    s = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', s)
    s = re.sub(r'<br />', ' ', s)
    s = re.sub(r'\'', ' ', s)

    return s

In [ ]:
# Clean data using the built in cleaner in gensim
train['text_clean1'] = train['text'].apply(lambda x: gensim.utils.simple_preprocess(x))
test['text_clean1'] = test['text'].apply(lambda x: gensim.utils.simple_preprocess(x))

train['text_clean2'] = train['text'].apply(lambda x: text_preprocessing(x))
test['text_clean2'] = test['text'].apply(lambda x: text_preprocessing(x))

train['text_clean3'] = list(map(clean_text, train['text']))
test['text_clean3'] = list(map(clean_text, test['text']))

## Feature engineering

In [ ]:
!pip install pyspellchecker
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=9a4f9d4192b84cd54ab7eaa93b4a747e2522512e10ba860cf972de68931707a8
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=2411be47e314844fef7d80587af4bb9d738c8a934d015a27652ba0c260119a29
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f

In [ ]:
def punctuations(text):
  count = 0
  for i in text:
    if i in string.punctuation:
      count +=1
  return count/(len(text))

In [ ]:
# spelling mistakes

# def spell3():
#     s = df_train['text'][1]
#     wordlist=s.split()
#     spell = SpellChecker()
#     print(list(spell.known(wordlist)))
#     print(list(spell.unknown(wordlist)))
#     amount_miss = len(list(spell.unknown(wordlist))) + len(list(spell.known(wordlist)))
#     print("Possible amount of misspelled words in the text:",amount_miss)
# spell3()

In [ ]:
train['punctuations'] = train['text'].apply(punctuations)
test['punctuations'] = train['text'].apply(punctuations)
train['punctuations'].sort_values(ascending = False) #.value_counts().reset_index().sort_values(by = 'punctuations', ascending = False)

485     0.461883
2570    0.356955
296     0.333333
2564    0.333333
1593    0.333333
          ...   
951     0.007485
343     0.006579
24      0.006309
1138    0.005800
2667    0.000000
Name: punctuations, Length: 2838, dtype: float64

## LDA

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod
    
def get_corpus(df):
    df['text_lda'] = df['text_clean2'].str.strip() #strip_newline(df.text)
    words = list(sent_to_words(df['text_lda']))
    words = remove_stopwords(words)
    bigram_mod = bigrams(words)
    bigram = [bigram_mod[r] for r in words]
    id2word = gensim.corpora.Dictionary(bigram)
    id2word.filter_extremes(no_below=10, no_above=0.35)
    id2word.compactify()
    corpus = [id2word.doc2bow(text) for text in bigram]
    return corpus, id2word, bigram

In [ ]:
stop_words = stopwords.words('english')
# stop_words.extend(['come','order','try','go','get','make','drink','plate','dish','restaurant','place',
#                   'would','really','like','great','service','came','got'])


train_corpus, train_id2word, bigram_train = get_corpus(train)
test_corpus, test_id2word, bigram_test = get_corpus(test)

In [ ]:
# from gensim.test.utils import common_corpus, common_dictionary
# from gensim.models import HdpModel

# hdp = HdpModel(train_corpus, train_id2word)
# hdp.get_topics()

In [ ]:

logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train = gensim.models.ldamulticore.LdaMulticore(
                           corpus=train_corpus,
                           num_topics= 10, #20, ################# number of subredit
                           id2word=train_id2word,
                           chunksize=100,
                           workers=7, # Num. Processing Cores - 1
                           passes=50,
                           eval_every = 1,
                           per_word_topics=True)
    lda_train.save('lda_train.model')

In [ ]:
import pyLDAvis.gensim_models
lda_display = pyLDAvis.gensim_models.prepare(lda_train, train_corpus, train_id2word, sort_topics=False)
pyLDAvis.display(lda_display)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
def get_bigram(df):
    """
    For the test data we only need the bigram data. This is a requirement due to 
    the shapes Gensim functions expect in the test-vector transformation below.
    With both these in hand, we can make the test corpus.
    """
    df['text'] = df.text.str.strip() 
    words = list(sent_to_words(df['text']))
    words = remove_stopwords(words)
    bigram = bigrams(words)
    bigram = [bigram[r] for r in words]
    return bigram

bigram_test = get_bigram(test)

test_corpus = [train_id2word.doc2bow(text) for text in bigram_test]

## Word2Vec embedding

In [ ]:
## word2vec embedding

w2v_X_train = train['text_clean2']
w2v_X_test = test['text_clean2'] 
w2v_y_train = train['label']
w2v_y_test = test['label']

# Train the word2vec model
w2v_model = gensim.models.Word2Vec(w2v_X_train,
                                   size=300,
                                   window=5,
                                   min_count=2)

w2v_words = set(w2v_model.wv.index2word) #index2word
w2v_X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in w2v_words])
                         for ls in w2v_X_train])
w2v_X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in w2v_words])
                         for ls in w2v_X_test])

w2v_X_train_vect_avg = []
for v in w2v_X_train_vect:
    if v.size:
        w2v_X_train_vect_avg.append(v.mean(axis=0))
    else:
        w2v_X_train_vect_avg.append(np.zeros(300, dtype=float))
        
w2v_X_test_vect_avg = []
for v in w2v_X_test_vect:
    if v.size:
        w2v_X_test_vect_avg.append(v.mean(axis=0))
    else:
        w2v_X_test_vect_avg.append(np.zeros(300, dtype=float))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


## gloVe embedding

In [ ]:
glove_x_train = train['text_clean2']
glove_y_train = train['label']
glove_x_test = test['text_clean2']
glove_y_test = test['label']

token = Tokenizer()
token.fit_on_texts(glove_x_train)
seq = token.texts_to_sequences(glove_x_train)

pad_seq = pad_sequences(seq,maxlen=300)

vocab_size = len(token.word_index)+1

In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(glove_x_train)

words_to_index = tokenizer.word_index

In [ ]:
def loading_embeddings():
    embeddings_index = {}
    f = open('glove.6B.100d.txt', encoding='utf8') # loading the file
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index

def prepare_embedding_matrix(word_index):
    embeddings_index = loading_embeddings()
    num_words = min(MAX_NB_WORDS, len(word_index))
    embedding_matrix = np.zeros((num_words + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        if i >= MAX_NB_WORDS:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix, num_words


def vectorizing_data(df):
    label_s = df['label'].tolist()
    l = list(set(label_s))
    l.sort()
    labels_index = dict([(j,i) for i, j in enumerate(l)])
    labels = [labels_index[i] for i in label_s]
    print('Found %s texts.' % len(df['text_clean2']))
    print('labels_index — ', labels_index)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(df['text_clean2'])
    sequences = tokenizer.texts_to_sequences(df['text_clean2'])
    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))
    df = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    labels = np.asarray(labels)

    return df, labels, word_index

In [ ]:
MAX_NB_WORDS = 100000
MAX_SEQUENCE_LENGTH = 300

df_train_for_glove = train[['text_clean2', 'label']]
df_test_for_glove = test[['text_clean2', 'label']]

glove_vecs_train, labels_train, word_index_train = vectorizing_data(df_train_for_glove)
glove_vecs_test, labels_test, word_index_test = vectorizing_data(df_test_for_glove)

Found 2838 texts.
labels_index —  {0: 0, 1: 1}
Found 11397 unique tokens.
Found 715 texts.
labels_index —  {0: 0, 1: 1}
Found 5599 unique tokens.


## FastText

In [ ]:
# fasttext.util.download_model('en', if_exists='ignore')  # English
ft_model = fasttext.load_model('cc.en.300.bin')
window_length = 500 # The amount of words we look at per example.
n_features = ft_model.get_dimension()

In [ ]:
def text_to_vector(text):
    """
    Given a string, normalizes it, then splits it into words and finally converts
    it to a sequence of word vectors.
    """
    words = text.split()
    window = words[-window_length:]
    
    x = np.zeros((window_length, n_features))

    for i, word in enumerate(window):
        x[i, :] = ft_model.get_word_vector(word).astype('float32')

    return x

def df_to_data(df):
    """
    Convert a given dataframe to a dataset of inputs for the NN.
    """
    x = np.zeros((len(df), window_length, n_features), dtype='float32')

    for i, comment in enumerate(df['text'].values):
        x[i, :] = text_to_vector(comment)

    return x

## Models

### Training

In [ ]:
def train_model(X,y):
  kf = KFold(5, shuffle=True, random_state=42)
  cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []

  for train_ind, val_ind in kf.split(X, y):
      # Assign CV IDX
      X_train, y_train = X[train_ind], y[train_ind]
      X_val, y_val = X[val_ind], y[val_ind]
      
      # Scale Data
      scaler = StandardScaler()
      X_train_scale = scaler.fit_transform(X_train)
      X_val_scale = scaler.transform(X_val)

      # Logisitic Regression
      lr = LogisticRegression(
          solver= 'liblinear',
          max_iter=1000,
          verbose=1,
          penalty='l2',
          C=5
          # fit_intercept=True
      ).fit(X_train_scale, y_train)


      y_pred = lr.predict(X_val_scale)
      cv_lr_f1.append(f1_score(y_val, y_pred))
      

  print(f'Logistic Regression Val f1: {np.mean(cv_lr_f1):.4f} +- {np.std(cv_lr_f1):.3f}')

  return lr

In [ ]:
y_train = np.array(train.label)

In [ ]:
# model - features only
train_vecs0 = []
for i in range(len(train)):
    topic_vec = []                                    
    topic_vec.extend([train.iloc[i]['lex_liwc_Tone']])                                   # Features
    topic_vec.extend([train.iloc[i]['lex_liwc_Clout']])                                  # Features
    topic_vec.extend([train.iloc[i]['lex_liwc_i']])                                      # Features
    train_vecs0.append(topic_vec)

X_train0 = np.array(train_vecs0)
lr0 = train_model(X_train0,y_train)

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Logistic Regression Val f1: 0.7535 +- 0.021


In [ ]:
# model - features + new
train_vecs1 = []
for i in range(len(train)):
    topic_vec = []                                    
    topic_vec.extend([train.iloc[i]['lex_liwc_Tone']])                                   # Features
    topic_vec.extend([train.iloc[i]['lex_liwc_Clout']])                                  # Features
    topic_vec.extend([train.iloc[i]['lex_liwc_i']])                                      # Features
    topic_vec.extend([train.iloc[i]['punctuations']])                                    # Features - new
    train_vecs1.append(topic_vec)

X_train1 = np.array(train_vecs1)
lr1 = train_model(X_train1, y_train)

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Logistic Regression Val f1: 0.7552 +- 0.018


In [ ]:
# model - features + new + LDA
train_vecs2 = []
for i in range(len(train)):
    top_topics = lda_train.get_document_topics(train_corpus[i], minimum_probability=0.0) # LDA
    topic_vec = [top_topics[i][1] for i in range(10)]                                    # LDA                                    
    topic_vec.extend([train.iloc[i]['lex_liwc_Tone']])                                   # Features
    topic_vec.extend([train.iloc[i]['lex_liwc_Clout']])                                  # Features
    topic_vec.extend([train.iloc[i]['lex_liwc_i']])                                      # Features
    topic_vec.extend([train.iloc[i]['punctuations']])                                    # Features - new
    train_vecs2.append(topic_vec)

X_train2 = np.array(train_vecs2)
lr2 = train_model(X_train2, y_train)

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Logistic Regression Val f1: 0.6910 +- 0.008


In [ ]:
# model - features + new + w2v
train_vecs3 = []
for i in range(len(train)):
    topic_vec = []                                    
    topic_vec.extend([train.iloc[i]['lex_liwc_Tone']])                                   # Features
    topic_vec.extend([train.iloc[i]['lex_liwc_Clout']])                                  # Features
    topic_vec.extend([train.iloc[i]['lex_liwc_i']])                                      # Features
    topic_vec.extend([train.iloc[i]['punctuations']])                                    # Features - new
    topic_vec.extend([w2v_X_train_vect_avg[i][j] for j in range(300)])                   # word2vec embedding
    train_vecs3.append(topic_vec)

X_train3 = np.array(train_vecs3)
lr3 = train_model(X_train3, y_train)

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Logistic Regression Val f1: 0.7591 +- 0.020


In [ ]:
# model - features + new + gloVe
train_vecs4 = []
for i in range(len(train)):
    topic_vec = []                                    
    topic_vec.extend([train.iloc[i]['lex_liwc_Tone']])                                   # Features
    topic_vec.extend([train.iloc[i]['lex_liwc_Clout']])                                  # Features
    topic_vec.extend([train.iloc[i]['lex_liwc_i']])                                      # Features
    topic_vec.extend([train.iloc[i]['punctuations']])                                    # Features - new
    topic_vec.extend([glove_vecs_train[i][j] for j in range(300)])                       # gloVe embedding
    train_vecs4.append(topic_vec)

X_train4 = np.array(train_vecs4)
lr4 = train_model(X_train4, y_train)

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Logistic Regression Val f1: 0.7393 +- 0.018


In [ ]:
# model - features + new + fastText
train_vecs5 = []
for i in range(len(train)):
    topic_vec = []                                    
    topic_vec.extend([train.iloc[i]['lex_liwc_Tone']])                                   # Features
    topic_vec.extend([train.iloc[i]['lex_liwc_Clout']])                                  # Features
    topic_vec.extend([train.iloc[i]['lex_liwc_i']])                                      # Features
    topic_vec.extend([train.iloc[i]['punctuations']])                                    # Features - new
    topic_vec.extend([fasttext_X_train_vect_avg[i][j] for j in range(300)])              # fastText embedding
    train_vecs5.append(topic_vec)

X_train5 = np.array(train_vecs5)
lr5 = train_model(X_train5, y_train)

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Logistic Regression Val f1: 0.7632 +- 0.010


### Evaluation

In [ ]:
def eval_model(X,y, lr):
  ss = StandardScaler()
  X = ss.fit_transform(X)

  y_pred_lr = lr.predict(X)

  f1_scored = f1_score(y, y_pred_lr,average='binary')
  precision_scored = precision_score(y, y_pred_lr,average='binary')
  recall_scored = recall_score(y, y_pred_lr,average='binary')
  
  print(f'F1 score: {f1_scored}')
  print(f'Precision: {precision_scored}')
  print(f'recall: {recall_scored}')

  return round(f1_scored, 4), round(precision_scored, 4), round(recall_scored, 4)

In [ ]:
y_test = np.array(test.label)

In [ ]:
# model - features only
name0 = 'features only'
test_vecs0 = []
for i in range(len(test)):
    topic_vec = []                                    
    topic_vec.extend([test.iloc[i]['lex_liwc_Tone']])                                   # Features
    topic_vec.extend([test.iloc[i]['lex_liwc_Clout']])                                  # Features
    topic_vec.extend([test.iloc[i]['lex_liwc_i']])                                      # Features
    test_vecs0.append(topic_vec)

X_test0 = np.array(test_vecs0)
f1_score0, precision_score0, recall_score0 = eval_model(X_test0,y_test, lr0)

F1 score: 0.7605263157894737
Precision: 0.7391304347826086
recall: 0.7831978319783198


In [ ]:
# model - features + new
name1 = 'features + new'
test_vecs1 = []
for i in range(len(test)):
    topic_vec = []                                    
    topic_vec.extend([test.iloc[i]['lex_liwc_Tone']])                                   # Features
    topic_vec.extend([test.iloc[i]['lex_liwc_Clout']])                                  # Features
    topic_vec.extend([test.iloc[i]['lex_liwc_i']])                                      # Features
    topic_vec.extend([test.iloc[i]['punctuations']])                                    # Features - new
    test_vecs1.append(topic_vec)

X_test1 = np.array(test_vecs1)
f1_score1, precision_score1, recall_score1 = eval_model(X_test1,y_test, lr1)

F1 score: 0.764629388816645
Precision: 0.735
recall: 0.7967479674796748


In [ ]:
# model - features + new + LDA
name2 = 'features + new + LDA'
test_vecs2 = []
for i in range(len(test)):
    top_topics = lda_train.get_document_topics(test_corpus[i], minimum_probability=0.0) # LDA
    topic_vec = [top_topics[i][1] for i in range(10)]                                   # LDA                                   
    topic_vec.extend([test.iloc[i]['lex_liwc_Tone']])                                   # Features
    topic_vec.extend([test.iloc[i]['lex_liwc_Clout']])                                  # Features
    topic_vec.extend([test.iloc[i]['lex_liwc_i']])                                      # Features
    topic_vec.extend([test.iloc[i]['punctuations']])                                    # Features - new
    test_vecs2.append(topic_vec)

X_test2 = np.array(test_vecs2)
f1_score2, precision_score2, recall_score2 = eval_model(X_test2,y_test, lr2)

F1 score: 0.6984126984126984
Precision: 0.6821705426356589
recall: 0.7154471544715447


In [ ]:
# model - features + new + w2v
name3 = 'features + new + w2v'
test_vecs3 = []
for i in range(len(test)):
    topic_vec = []                                    
    topic_vec.extend([test.iloc[i]['lex_liwc_Tone']])                                   # Features
    topic_vec.extend([test.iloc[i]['lex_liwc_Clout']])                                  # Features
    topic_vec.extend([test.iloc[i]['lex_liwc_i']])                                      # Features
    topic_vec.extend([test.iloc[i]['punctuations']])                                    # Features - new
    topic_vec.extend([w2v_X_test_vect_avg[i][j] for j in range(300)])                   # word2vec embedding
    test_vecs3.append(topic_vec)

X_test3 = np.array(test_vecs3)
f1_score3, precision_score3, recall_score3 = eval_model(X_test3,y_test, lr3)

F1 score: 0.7735099337748346
Precision: 0.7564766839378239
recall: 0.7913279132791328


In [ ]:
# model - features + new + gloVe
name4 = 'features + new + gloVe'
test_vecs4 = []
for i in range(len(test)):
    topic_vec = []                                    
    topic_vec.extend([test.iloc[i]['lex_liwc_Tone']])                                   # Features
    topic_vec.extend([test.iloc[i]['lex_liwc_Clout']])                                  # Features
    topic_vec.extend([test.iloc[i]['lex_liwc_i']])                                      # Features
    topic_vec.extend([test.iloc[i]['punctuations']])                                    # Features - new
    topic_vec.extend([glove_vecs_test[i][j] for j in range(300)])                       # gloVe embedding
    test_vecs4.append(topic_vec)

X_test4 = np.array(test_vecs4)
f1_score4, precision_score4, recall_score4 = eval_model(X_test4,y_test, lr4)

F1 score: 0.8015873015873015
Precision: 0.7829457364341085
recall: 0.8211382113821138


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


In [ ]:
# model - features + new + fastText
name5 = 'features + new + fastText'
test_vecs5 = []
for i in range(len(test)):
    topic_vec = []                                    
    topic_vec.extend([test.iloc[i]['lex_liwc_Tone']])                                   # Features
    topic_vec.extend([test.iloc[i]['lex_liwc_Clout']])                                  # Features
    topic_vec.extend([test.iloc[i]['lex_liwc_i']])                                      # Features
    topic_vec.extend([test.iloc[i]['punctuations']])                                    # Features - new
    topic_vec.extend([fattext_X_test_vect_avg[i][j] for j in range(300)])               # fastText embedding
    test_vecs5.append(topic_vec)

X_test5 = np.array(test_vecs5)
f1_score5, precision_score5, recall_score5 = eval_model(X_test5,y_test, lr5)

F1 score: 0.8348993288590604
Precision: 0.8271276595744681
recall: 0.8428184281842819


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


# BERT

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 6.9 MB/s 
     |████████████████████████████████| 120 kB 90.3 MB/s 
     |████████████████████████████████| 6.6 MB 56.7 MB/s 


In [ ]:
train_text = train['text']
train_labels = train['label']

test_text = test['text']
test_labels = test['label']


train_text, val_text, train_labels, val_labels = train_test_split(train['text'], train['label'], 
                                                                random_state=2018, 
                                                                test_size=0.15, 
                                                                stratify=train['label'])

In [ ]:
def text_preprocessing2(text):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """

    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    # text = text.lower()

    return text

In [ ]:
from transformers import BertTokenizer
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification


# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing2(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [ ]:
# Specify `MAX_LEN`
MAX_LEN = 300
# Run function `preprocessing_for_bert` on the train set and the validation set
print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(train_text)
val_inputs, val_masks = preprocessing_for_bert(val_text)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Tokenizing data...


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert other data types to torch.Tensor
train_labelss = torch.tensor(train_labels.to_numpy())
val_labelss = torch.tensor(val_labels.to_numpy())

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 16

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labelss)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_masks, val_labelss)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [ ]:
%%time
import torch
import torch.nn as nn
from transformers import BertModel

# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 2

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

CPU times: user 44 µs, sys: 0 ns, total: 44 µs
Wall time: 47 µs


In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [ ]:
import random
import time

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)
            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [ ]:
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
train(bert_classifier, train_dataloader, val_dataloader, epochs=2, evaluation=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a

Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   0.697030   |     -      |     -     |   13.26  
   1    |   40    |   0.691691   |     -      |     -     |   9.76   
   1    |   60    |   0.677739   |     -      |     -     |   9.75   
   1    |   80    |   0.603047   |     -      |     -     |   9.74   
   1    |   100   |   0.582703   |     -      |     -     |   9.74   
   1    |   120   |   0.590601   |     -      |     -     |   9.74   
   1    |   140   |   0.533755   |     -      |     -     |   9.74   
   1    |   150   |   0.545170   |     -      |     -     |   4.77   
----------------------------------------------------------------------
   1    |    -    |   0.620398   |  0.527350  |   74.03   |   80.72  
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elaps

In [ ]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs

In [ ]:
# Compute predicted probabilities on the test set
# probs = bert_predict(bert_classifier, val_dataloader)

In [ ]:
# Concatenate the train set and the validation set
full_train_data = torch.utils.data.ConcatDataset([train_data, val_data])
full_train_sampler = RandomSampler(full_train_data)
full_train_dataloader = DataLoader(full_train_data, sampler=full_train_sampler, batch_size=16)

# Train the Bert Classifier on the entire training data
set_seed(42)
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
train(bert_classifier, full_train_dataloader, epochs=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a

Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   0.634572   |     -      |     -     |   10.27  
   1    |   40    |   0.468481   |     -      |     -     |   9.76   
   1    |   60    |   0.456016   |     -      |     -     |   9.74   
   1    |   80    |   0.410451   |     -      |     -     |   9.75   
   1    |   100   |   0.411934   |     -      |     -     |   9.75   
   1    |   120   |   0.440629   |     -      |     -     |   9.74   
   1    |   140   |   0.419315   |     -      |     -     |   9.76   
   1    |   160   |   0.420392   |     -      |     -     |   9.73   
   1    |   177   |   0.448730   |     -      |     -     |   7.98   
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
-------------------------------------------------------------------

In [ ]:
# Run `preprocessing_for_bert` on the test set
print('Tokenizing data...')
test_inputs, test_masks = preprocessing_for_bert(test_text)

# Create the DataLoader for our test set
test_dataset = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=32)

Tokenizing data...


In [ ]:
# Compute predicted probabilities on the test set
probs = bert_predict(bert_classifier, test_dataloader)
preds = np.argmax(probs, axis=1)
# Get predictions from the probabilities
# threshold = 0.9
# preds = np.where(probs[:, 1] > threshold, 1, 0)

# Number of tweets predicted non-negative
# print("Number of tweets predicted non-negative: ", preds.sum())

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

y_true = np.array(test_labels)
y_pred = np.array(preds)

precision_recall_fscore_support(y_true, y_pred, average='macro')

(0.8263062720091368, 0.8222425865877155, 0.822802586858006, None)